本代码用于检验纯随机噪声下的情况

实例：Debugging

状态：已圆满完成任务


In [1]:
import argparse
import datetime
import glob
import os
from pathlib import Path
from test import repeat_eval_ckpt, eval_single_ckpt
# from noise import add_noise_to_weights
from hardware_noise.weight_mapping_finalv import weight_mapping as add_noise_to_weights

import numba
import logging
import time

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

import torch
import torch.distributed as dist
import torch.nn as nn
from tensorboardX import SummaryWriter

from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.datasets import build_dataloader
from pcdet.models_multinomial_finalv import build_network, model_fn_decorator
from pcdet.utils import common_utils
from train_utils.optimization import build_optimizer, build_scheduler
from train_utils.train_utils import train_model, model_save
from eval_utils import eval_utils_multinomial_finalv as eval_utils
import numpy as np
from utilis_gpu import get_default_device, to_device


def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='./cfgs/kitti_models/pointpillar_bayes.yaml', \
                        help='specify the config for training')
    # sunqiao/OpenPCDet/tools/cfgs/kitti_models/pointpillar_bayes.yaml
    parser.add_argument('--batch_size', type=int, default=8, required=False, help='batch size for training')
    parser.add_argument('--epochs', type=int, default=1, required=False, help='number of epochs to train for')
    parser.add_argument('--workers', type=int, default=32, help='number of workers for dataloader')
    parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    parser.add_argument('--ckpt', type=str, default='./checkpoint_epoch_80_bayes.pth', help='checkpoint to start from')
    # ./checkpoint_epoch_80.pth
    parser.add_argument('--pretrained_model', type=str, default=False, help='pretrained_model')
    parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    parser.add_argument('--tcp_port', type=int, default=18888, help='tcp port for distrbuted training')
    parser.add_argument('--sync_bn', action='store_true', default=False, help='whether to use sync bn')
    parser.add_argument('--fix_random_seed', action='store_true', default=True, help='')
    parser.add_argument('--ckpt_save_interval', type=int, default=1, help='number of training epochs')
    parser.add_argument('--local_rank', type=int, default=0, help='local rank for distributed training')
    parser.add_argument('--max_ckpt_save_num', type=int, default=81, help='max number of saved checkpoint')
    parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER,
                        help='set extra config keys if needed')

    parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    parser.add_argument('--start_epoch', type=int, default=0, help='')
    parser.add_argument('--save_to_file', action='store_true', default=False, help='')

    args = parser.parse_known_args()[0]

    cfg_from_yaml_file(args.cfg_file, cfg)
    cfg.TAG = Path(args.cfg_file).stem
    cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

    if args.set_cfgs is not None:
        cfg_from_list(args.set_cfgs, cfg)

    return args, cfg

class Opt():
    def __init__(self, file):
        self.file = file
        
    def opt_function(self, pp1, pp2, pp3, pp4):
        
        global p1
        global p2
        global p3
        global p4
        
        p1 = pp1
        p2 = pp2
        p3 = pp3
        p4 = pp4
        
        print("=============")
        print(p1, p2, p3, p4)
        print("=============")

        global best_accu
        
        with_training = True  # False
        if with_training == True:
            # p1 = round(p1, 2)
            # p2 = round(p2, 2)
            # p3 = round(p3, 2)
            # p4 = round(p4, 2)
            
            train_set, train_loader, train_sampler = build_dataloader(
                dataset_cfg=cfg.DATA_CONFIG,
                class_names=cfg.CLASS_NAMES,
                batch_size=args.batch_size,
                dist=dist_train, workers=args.workers,
                logger=logger,
                training=True,
                merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
                total_epochs=args.epochs
            )

            model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), 
                                p1=p1, 
                                p2=p2, 
                                p3=p3, 
                                p4=p4, 
                                dataset=train_set)
            # print(model.state_dict())

            if args.sync_bn:
                model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
            model.cuda()

            optimizer = build_optimizer(model, cfg.OPTIMIZATION)

            # # load checkpoint if it is possible
            start_epoch = it = 0
            last_epoch = -1
            if args.pretrained_model is True:
                model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)


            model.train()  # before wrap to DistributedDataParallel to support fixed some parameters
            if dist_train:
                model = nn.parallel.DistributedDataParallel(model, device_ids=[cfg.LOCAL_RANK % torch.cuda.device_count()])
            logger.info(model)

            lr_scheduler, lr_warmup_scheduler = build_scheduler(
                optimizer, total_iters_each_epoch=len(train_loader), total_epochs=args.epochs,
                last_epoch=last_epoch, optim_cfg=cfg.OPTIMIZATION
            )

        #     # -----------------------start training---------------------------
            logger.info('**********************Start training %s/%s(%s)**********************'
                        % (cfg.EXP_GROUP_PATH, cfg.TAG, args.extra_tag))

            train_model(
                model,
                optimizer,
                train_loader,
                model_func=model_fn_decorator(),
                lr_scheduler=lr_scheduler,
                optim_cfg=cfg.OPTIMIZATION,
                start_epoch=start_epoch,
                total_epochs=args.epochs,
                start_iter=it,
                rank=cfg.LOCAL_RANK,
                tb_log=tb_log,
                ckpt_save_dir=ckpt_dir,
                train_sampler=train_sampler,
                lr_warmup_scheduler=lr_warmup_scheduler,
                ckpt_save_interval=args.ckpt_save_interval,
                max_ckpt_save_num=args.max_ckpt_save_num,
                merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch
            )

        test_set, test_loader, sampler = build_dataloader(
                                        dataset_cfg=cfg.DATA_CONFIG,
                                        class_names=cfg.CLASS_NAMES,
                                        batch_size=args.batch_size,
                                        dist=dist_train, workers=args.workers, logger=logger, training=False
                                    )


        model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), p1=0.11, p2=0.11, p3=0.11, p4=0.11, dataset=test_set)
        model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)
        # model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)
        
        optimizer = build_optimizer(model, cfg.OPTIMIZATION)

        if dist_train: 
            model = model.module

        # if args.pretrained_model is True:
        #     model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)

        # model.load_params_from_file(filename='./checkpoint_epoch_80.pth', logger=logger, to_cpu=dist_train)
        model.cuda()
        
        global n
        n += 1
        
        ckpt_pth = save_path+'multinomial_iter{}-{}-{}-{}-{}'.format(n, p1, p2, p3, p4)
        ckpt_name = ckpt_pth+'.pth'

        if cfg.LOCAL_RANK == 0:
            model_save(model, ckpt_pth, optimizer, args.epochs, args.epochs)

        logger.info('**********************End training**********************')

        time.sleep(1)


        f = open(save_path+'result.txt', "a+")
        f.write('----------------{}-{}-{}-{}---------------\n'.format(p1, p2, p3, p4))
        f.close()


        acc1, ret_dict = eval_utils.eval_simple(p1, p2, p3, p4, None, None, None, 
                                                n, cfg, model, test_loader, logger, 
                                                save_path, dist_test=dist_train, 
                                                save_to_file=args.save_to_file, 
                                                result_dir=eval_output_dir)
        print("----------")
        print(acc1)
        print("----------")


        logger.info('**********************End evaluation**********************')

        return acc1




if __name__ == '__main__':

    torch.cuda.set_device(0)
    # best_accu = 0

    args, cfg = parse_config()
    if args.launcher == 'none':
        dist_train = False
        total_gpus = 1
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.system('nvidia-smi -q -d Memory |grep -A4 GPU|grep Free >tmp')
        memory_gpu = [int(x.split()[2]) for x in open('tmp', 'r').readlines()]
        print('Using GPU:' + str(np.argmax(memory_gpu)))
        os.environ["CUDA_VISIBLE_DEVICES"] = str(np.argmax(memory_gpu))
        os.system('rm tmp')
    else:
        total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
            args.tcp_port, args.local_rank, backend='nccl'
        )
        dist_train = True

    if args.batch_size is None:
        args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
    else:
        assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
        args.batch_size = args.batch_size // total_gpus

    args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

    if args.fix_random_seed:
        common_utils.set_random_seed(666)

    output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / 'bayes' / args.extra_tag
    ckpt_dir = output_dir / 'ckpt'
    output_dir.mkdir(parents=True, exist_ok=True)
    ckpt_dir.mkdir(parents=True, exist_ok=True)

    save_path = './save_path/multinomial/'#/bayes/pointpillar/'+time.strftime('%m%d-%H%M',time.localtime(time.time()))+'/'

    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True) 

    logger = common_utils.create_logger(save_path+'log.txt', rank=cfg.LOCAL_RANK)

    file = open(save_path+'result.txt','w')
    file.write('results\n')
    file.close()


    if dist_train:
        logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))
    for key, val in vars(args).items():
        logger.info('{:16} {}'.format(key, val))
    log_config_to_file(cfg, logger=logger)
    if cfg.LOCAL_RANK == 0:
        os.system('cp %s %s' % (args.cfg_file, output_dir))

    tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

    eval_output_dir = output_dir / 'eval' / 'eval_with_train'
    eval_output_dir.mkdir(parents=True, exist_ok=True)
    args.start_epoch = max(args.epochs - 10, 0)  # Only evaluate the last 10 epochs
    
    
    print("=============")
    
    p1 = 0.23
    p2 = 0.14
    p3 = 0.68
    p4 = 0.36
    
    print(p1, p2, p3, p4)
    print("=============")
    

    '''Bayesian Training'''
    bayes_train = False
    if bayes_train:
        logger.info('----------------Start Bayes Optimization----------------')
        for f_name in os.listdir('hardware_noise'):

            opt= Opt(file)
            opt_function = opt.opt_function

            # Bounded region of parameter space
            pbounds = {'pp1': (0.1, 0.9), 'pp2': (0.1, 0.9), 'pp3': (0.1, 0.9), 'pp4': (0.1, 0.9)}


            optimizer_bayes = BayesianOptimization(
                f=opt_function,
                pbounds=pbounds,
                verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
                random_state=1,
            )
            optimizer_bayes.probe(
                params={'pp1': 0.11, 'pp2': 0.11, 'pp3': 0.11, 'pp4': 0.11},
                lazy=True,
            )

            logger_bayes = JSONLogger(path=save_path+"logs2.json")
            optimizer_bayes.subscribe(Events.OPTIMIZATION_STEP, logger_bayes)

            n = 0
            optimizer_bayes.maximize(
                init_points=3,
                n_iter=80,
            )
        logger.info('----------------End Bayes Optimization----------------')
            
            
    '''Test on Noises'''
    
    def add_noise_to_weights(mean, std, model):
        gassian_kernel = torch.distributions.Normal(mean, std)
        with torch.no_grad():
            for param in model.parameters():
                param.mul_(torch.exp(gassian_kernel.sample(param.size())).cuda())
                
    
    logger.info('**********************Start evaluation**********************')    
    test_set, test_loader, sampler = build_dataloader(
                                    dataset_cfg=cfg.DATA_CONFIG,
                                    class_names=cfg.CLASS_NAMES,
                                    batch_size=args.batch_size,
                                    dist=dist_train, workers=args.workers, logger=logger, training=False
                                )

    model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), p1=0.11, p2=0.11, p3=0.11, p4=0.11, dataset=test_set)
    model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)
    
    optimizer = build_optimizer(model, cfg.OPTIMIZATION)

    model.cuda()

    # ckpt_pth = save_path+'bayes_model-{}-{}-{}-{}'.format(p1, p2, p3, p4)
    # ckpt_name = ckpt_pth+'.pth'
    # model_save(model, ckpt_pth, optimizer, args.epochs, args.epochs)

    time.sleep(1)
    
    file2usability = np.load('./hardware_noise/file2usability.npy', allow_pickle=True).item()
    file2sigma = np.load('./hardware_noise/file2sigma.npy', allow_pickle=True).item()
    
    # hw_data_files = sorted(os.listdir('./hardware_noise/hardware_data/'))
    file2ap_dict = {}
    N = 1        
                
                
    for sigma, f_name in sorted(zip(file2sigma.values(), file2sigma.keys())):
        print(sigma, f_name)
        usability = file2usability[f_name]
    # for f_name in sorted(hw_data_files):
        if f_name.endswith('xlsx'):
            file2ap_dict[f_name] = {}
    
    for sigma in np.linspace(1e-12, 0.5, 21):
        # print(f_name)
        for n in range(N):
            print('file:{}, evaluate-{}'.format(f_name, n))
            model.load_params_from_file(filename=args.ckpt, to_cpu=dist, logger=logger)
            add_noise_to_weights('./hardware_noise/hardware_data/'+f_name, model, device='cuda')
            # add_noise_to_weights(mean=0, std=sigma, model=model)

            usability = file2usability[f_name]
            sigma = file2sigma[f_name]

            acc1, ret_dict = eval_utils.eval_simple(p1, p2, p3, p4, f_name, usability, sigma, n, cfg, model, test_loader, logger, save_path=None, dist_test=dist_train,
                                            save_to_file=args.save_to_file, result_dir=eval_output_dir
                                        )                
            print(ret_dict)
            file2ap_dict[f_name][n] = ret_dict
                
                

    logger.info('**********************End evaluation**********************')    

    


print("=======end========")


2023-07-06 21:06:32,000   INFO  cfg_file         ./cfgs/kitti_models/pointpillar_bayes.yaml
2023-07-06 21:06:32,002   INFO  batch_size       8
2023-07-06 21:06:32,002   INFO  epochs           1
2023-07-06 21:06:32,003   INFO  workers          32
2023-07-06 21:06:32,003   INFO  extra_tag        default
2023-07-06 21:06:32,004   INFO  ckpt             ./checkpoint_epoch_80_bayes.pth
2023-07-06 21:06:32,004   INFO  pretrained_model False
2023-07-06 21:06:32,004   INFO  launcher         none
2023-07-06 21:06:32,005   INFO  tcp_port         18888
2023-07-06 21:06:32,005   INFO  sync_bn          False
2023-07-06 21:06:32,005   INFO  fix_random_seed  True
2023-07-06 21:06:32,006   INFO  ckpt_save_interval 1
2023-07-06 21:06:32,006   INFO  local_rank       0
2023-07-06 21:06:32,007   INFO  max_ckpt_save_num 81
2023-07-06 21:06:32,007   INFO  merge_all_iters_to_one_epoch False
2023-07-06 21:06:32,007   INFO  set_cfgs         None
2023-07-06 21:06:32,008   INFO  max_waiting_mins 0
2023-07-06 21:

Using GPU:0
0.23 0.14 0.68 0.36


/home/pai/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-07-06 21:06:33,699   INFO  ==> Loading parameters from checkpoint ./checkpoint_epoch_80_bayes.pth to CPU
2023-07-06 21:06:33,732   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+02ac3e1
2023-07-06 21:06:33,903   INFO  ==> Done (loaded 127/127)
2023-07-06 21:06:34,913   INFO  ==> Loading parameters from checkpoint ./checkpoint_epoch_80_bayes.pth to CPU
2023-07-06 21:06:34,930   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+02ac3e1
2023-07-06 21:06:35,013   INFO  ==> Done (loaded 127/127)


7.757076617433052e-06 I-V_data_1.8um_length_50nm_diameter_NA_third_etch_20min_Pb_ED_3h_180C_MAI_no_200nm_Ag_memory_6V_carbon_paste.xlsx
0.002659952221105318 I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_180+210C_FAI_PMMA_200nm_Ag_memory_6V_silver_paste.xlsx
0.0030656818966983616 I-V_data_1.2um_length_250nm_diameter_NA_third_etch_15min_Pb_ED_3h_180C_MAI_PMMA_600nm_Ag_memory_6V_carbon_paste.xlsx
0.004893866123810629 I-V_data_2um_length_250nm_diameter_NA_third_etch_15min_Pb_ED_3h_180C_MAI_no_200nm_Ag_memory_6V_carbon_paste.xlsx
0.006552443852351945 I-V_data_1.2um_length_200nm_diameter_NA_third_etch_15min_Pb_ED_10h_210C_MABr_PMMA_200nm_Ag_memory_6V_silver_paste.xlsx
0.006588873912809541 I-V_data_1um_length_200nm_diameter_NA_third_etch_10min_Pb_ED_1h_180C_MAI_no_100nm_Ag_memory_8V_carbon_paste.xlsx
0.006720265700331257 I-V_data_1.2um_length_250nm_diameter_NA_third_etch_15min_Pb_ED_3h_180C_MAI_no_600nm_Ag_memory_6V_carbon_paste.xlsx
0.007282609885080162 I-V_data_1um_leng

eval: 100%|██████████| 472/472 [01:46<00:00,  4.43it/s, recall_0.3=(0, 15935) / 17558]
2023-07-06 21:08:22,014   INFO  Generate label finished(sec_per_example: 0.0283 second).
2023-07-06 21:08:22,016   INFO  recall_roi_0.3: 0.000000
2023-07-06 21:08:22,016   INFO  recall_rcnn_0.3: 0.907564
2023-07-06 21:08:22,016   INFO  recall_roi_0.5: 0.000000
2023-07-06 21:08:22,017   INFO  recall_rcnn_0.5: 0.803793
2023-07-06 21:08:22,017   INFO  recall_roi_0.7: 0.000000
2023-07-06 21:08:22,017   INFO  recall_rcnn_0.7: 0.475510
2023-07-06 21:08:22,023   INFO  Average predicted number of objects(3769 samples): 43.847
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 96 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 125 will likely result in GPU under-utilization due to low occupancy.
  warn(

...

{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.05023351178949766, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.006435812734935642, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0002847704749971523, 'Car_aos/easy_R40': 0.0012263222641252175, 'Car_aos/moderate_R40': 0.002247172844316258, 'Car_aos/hard_R40': 0.005282339962218075, 'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 2.856979601165648e-05, 'Car_3d/hard_R40': 3.4885505770062654e-05, 'Car_bev/easy_R40': 0.0001149493237695496, 'Car_bev/moderate_R40': 0.0001342420265605904, 'Car_bev/hard_R40': 0.0002255183700820887, 'Car_image/easy_R40': 0.0022615832888134937, 'Car_image/moderate_R40': 0.004376642056809319, 'Car_image/hard_R40': 0.010193309379191403, 'Pedestrian_aos/easy_R40': 0.0004951564400583374, 'Pedestrian_aos/moderate_R40': 0.0007035533137965491, 'Pedestrian_aos/hard_R40': 0.0017004303617012915, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestria

eval: 100%|██████████| 472/472 [01:48<00:00,  4.36it/s, recall_0.3=(0, 24) / 17558]
2023-07-06 21:51:32,742   INFO  Generate label finished(sec_per_example: 0.0287 second).
2023-07-06 21:51:32,744   INFO  recall_roi_0.3: 0.000000
2023-07-06 21:51:32,744   INFO  recall_rcnn_0.3: 0.001367
2023-07-06 21:51:32,744   INFO  recall_roi_0.5: 0.000000
2023-07-06 21:51:32,744   INFO  recall_rcnn_0.5: 0.000342
2023-07-06 21:51:32,745   INFO  recall_roi_0.7: 0.000000
2023-07-06 21:51:32,745   INFO  recall_rcnn_0.7: 0.000000
2023-07-06 21:51:32,749   INFO  Average predicted number of objects(3769 samples): 13.724
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 36 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(Numb

{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.001366898279986331, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 0.00034172456999658277, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.0, 'Car_aos/moderate_R40': 0.00022204675887491565, 'Car_aos/hard_R40': 0.00022204675887491565, 'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 0.0, 'Car_3d/hard_R40': 0.0, 'Car_bev/easy_R40': 0.0, 'Car_bev/moderate_R40': 0.0, 'Car_bev/hard_R40': 0.0, 'Car_image/easy_R40': 0.0, 'Car_image/moderate_R40': 0.00043786671337244935, 'Car_image/hard_R40': 0.00043786671337244935, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pedestrian_image/moderate_R40': 0.0, 'Pedestrian_image/hard_R40': 0.0, 'Cyclist_aos/easy_R40': 

eval: 100%|██████████| 472/472 [01:50<00:00,  4.26it/s, recall_0.3=(0, 2) / 17558]
2023-07-06 21:53:32,031   INFO  Generate label finished(sec_per_example: 0.0294 second).
2023-07-06 21:53:32,031   INFO  recall_roi_0.3: 0.000000
2023-07-06 21:53:32,032   INFO  recall_rcnn_0.3: 0.000114
2023-07-06 21:53:32,032   INFO  recall_roi_0.5: 0.000000
2023-07-06 21:53:32,032   INFO  recall_rcnn_0.5: 0.000057
2023-07-06 21:53:32,033   INFO  recall_roi_0.7: 0.000000
2023-07-06 21:53:32,033   INFO  recall_rcnn_0.7: 0.000000
2023-07-06 21:53:32,040   INFO  Average predicted number of objects(3769 samples): 9.408
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 24 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/pai/lib/python3.9/site-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 25 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaP

{'recall/roi_0.3': 0.0, 'recall/rcnn_0.3': 0.00011390818999886092, 'recall/roi_0.5': 0.0, 'recall/rcnn_0.5': 5.695409499943046e-05, 'recall/roi_0.7': 0.0, 'recall/rcnn_0.7': 0.0, 'Car_aos/easy_R40': 0.0, 'Car_aos/moderate_R40': 0.0006114019022923656, 'Car_aos/hard_R40': 0.0006114019022923656, 'Car_3d/easy_R40': 0.0, 'Car_3d/moderate_R40': 0.0, 'Car_3d/hard_R40': 0.0, 'Car_bev/easy_R40': 0.0, 'Car_bev/moderate_R40': 0.0, 'Car_bev/hard_R40': 0.0, 'Car_image/easy_R40': 0.0, 'Car_image/moderate_R40': 0.0006132711885195633, 'Car_image/hard_R40': 0.0006132711885195633, 'Pedestrian_aos/easy_R40': 0.0, 'Pedestrian_aos/moderate_R40': 0.0, 'Pedestrian_aos/hard_R40': 0.0, 'Pedestrian_3d/easy_R40': 0.0, 'Pedestrian_3d/moderate_R40': 0.0, 'Pedestrian_3d/hard_R40': 0.0, 'Pedestrian_bev/easy_R40': 0.0, 'Pedestrian_bev/moderate_R40': 0.0, 'Pedestrian_bev/hard_R40': 0.0, 'Pedestrian_image/easy_R40': 0.0, 'Pedestrian_image/moderate_R40': 0.0, 'Pedestrian_image/hard_R40': 0.0, 'Cyclist_aos/easy_R40': 0.0

In [2]:
np.linspace(0,1,21)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [3]:
# pip install -r ../requirements.txt

In [4]:
# cd ..

In [5]:
# !python setup.py develop

In [6]:
# cd tools

In [7]:
# pip install -r ../requirements.txt

In [8]:
# ../output/cfgs/kitti_models/bayes/default/ckpt/checkpoint_epoch_33.pth